In [1]:
import time
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import os
import trimesh

import igl
from meshplot import plot, subplot, interact
import meshplot as mp

In [2]:
n_meshes = 0
model = None
path_ = "../data"

In [3]:
def load_data(shape=(2319, 3), do_scale=True, subtract_mean=False):
    global n_meshes, path_
    face_a = None
    data_set = []
    scale = 1.0
    n_meshes = 0
    for f in os.listdir(path_):
        mesh = trimesh.load(os.path.join(path_, f), process=False)
        v = np.array(mesh.vertices, np.float32)
        face_a = np.array(mesh.faces, np.int32)
        if shape == (2400, 3):
            v = np.vstack([v, np.zeros((81, 3), np.float32)])

        data_set.append(v)
        n_meshes += 1

    data_set = np.array(data_set)

    if do_scale:
        scale = np.max(data_set)
        data_set = data_set / scale

    mean_a = np.mean(data_set, axis=(0))

    if subtract_mean:
        data_set -= mean_a

    return data_set, face_a, mean_a, scale

In [4]:
encoder = models.load_model("./enc_nn2_noscale_nomean")
decoder = models.load_model("./dec_nn2_noscale_nomean")
shape = (22779, 3)

In [5]:
data_set, face_a, mean_a, scale = load_data(shape=shape, do_scale=False, subtract_mean=False)

In [6]:
@mp.interact(m=(0, n_meshes - 1))
def display_mesh(m):
    global data_set, face_a
    a = data_set[m] + mean_a
    v = np.zeros_like(a)
    v[:,0] = a[:, 0]
    v[:,1] = a[:, 2]
    v[:,2] = -a[:, 1]
    f = face_a
    p = plot(v, f, shading={"bbox": True})


interactive(children=(IntSlider(value=54, description='m', max=109), Output()), _dom_classes=('widget-interact…

In [8]:
@mp.interact(m=(0, n_meshes - 1))
def reconstruct_mesh(m):
    global data_set, face_a, encoder, decoder
    a = data_set[m]
    #print(a)
    
    o = np.zeros_like(a[:22779])
    o[:22779,0] = a[:22779, 0]
    o[:22779,1] = a[:22779, 2]
    o[:22779,2] = -a[:22779, 1]
    
    a = np.expand_dims(a, axis=0)
    e = encoder(a)
    a = decoder(e)
    b = np.squeeze(a)
    a = b
    v = np.zeros_like(a[:22779])
    #print(a)
    
    v[:22779,0] = a[:22779, 0]
    v[:22779,1] = a[:22779, 2]
    v[:22779,2] = -a[:22779, 1]
    f = face_a
    p = plot(v, f, shading={"bbox": True})
    p = plot(o, f, shading={"bbox": True})
    

interactive(children=(IntSlider(value=54, description='m', max=109), Output()), _dom_classes=('widget-interact…

In [10]:
@mp.interact(s0=(-10.0, 10.0), s1=(-10.0, 10.0), 
             s2=(-10.0, 10.0), s3=(-10.0, 10.0), 
             s4=(-10.0, 10.0), s5=(-10.0, 10.0),
             s6=(-10.0, 10.0), s7=(-10.0, 10.0)
            )
def reconstruct_mesh(s0, s1, s2, s3, s4, s5, s6, s7):
    global encoder, decoder
    q = np.array([s0, s1, s2, s3, s4, s5, s6, s7])
    q = np.expand_dims(q, axis=0)
    
    a = decoder(q)
    a = np.squeeze(a)
    v = np.zeros_like(a[:22779])
    #print(a)
    
    v[:22779,0] = a[:22779, 0]
    v[:22779,1] = a[:22779, 2]
    v[:22779,2] = -a[:22779, 1]
    f = face_a
    p = plot(v, f, shading={"bbox": True})
    

interactive(children=(FloatSlider(value=0.0, description='s0', max=10.0, min=-10.0), FloatSlider(value=0.0, de…